In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import random

In [2]:
filename = 'C:/Users/STECCAGIUSEPPE/OneDrive - CNR/IASI/Research/2024/StochasticWasteCollection/WMS_Customers.shp'
gdf = gpd.read_file(filename)
gdf = gdf.set_index('id')
gdf

,name,type,demand,geometry
id,,,,
1,schoolA,school,10,POINT (12.54296 41.95204)
0,depot,depot,0,POINT (12.54540 41.95585)
7,bar1,bar,10,POINT (12.54635 41.94994)
3,home2,home,10,POINT (12.54868 41.95490)
2,home1,home,10,POINT (12.53693 41.94905)
4,home3,home,10,POINT (12.54979 41.94693)
5,home4,home,10,POINT (12.55721 41.94709)
6,home5,home,10,POINT (12.55156 41.96201)
8,bar2,bar,10,POINT (12.54197 41.95074)


In [3]:
minY = 12.53 #ovest
maxY = 12.58 #est
minX = 41.93 #sud
maxX = 41.97 #nord

G = ox.graph_from_bbox(maxX,minX, maxY, minY, network_type='drive')

In [4]:
df_links = pd.DataFrame(columns=['i', 'j', 'd', 'c', 't'])
for index1, row1 in gdf.iterrows():
    for index2, row2 in gdf.iterrows():
        i = index1
        j = index2
        orig = ox.distance.nearest_nodes(G, row1['geometry'].x, row1['geometry'].y)
        dest = ox.distance.nearest_nodes(G, row2['geometry'].x, row2['geometry'].y)
        dist = nx.shortest_path_length(G, orig, dest,weight= 'length')
        cost = dist
        kmh = 30
        t = (dist / 1000)*(60/kmh)
        #print (dist, t)
        df_links.loc[len(df_links)] = [i,j,dist,cost,t]
#gdf.loc[1]['geometry'].x

In [5]:
df_links.to_excel('real_instance_links.xlsx', index=None)

In [6]:
gdf.to_excel('real_instance_nodes.xlsx')

In [7]:
random_bounds = {'extralow' : (0, 0.1), 'low' : (0.1, 0.2), 'mean' : (0.2, 0.3), 'high' : (0.3, 0.4), 'extrahigh' : (0.4, 0.5)}
time_slots = [(8, 12), (12, 16), (16, 20), (20, 24)]
demand_type = {'home' : 'mean', 'school' : 'extralow', 'restaurant' : 'high', 'bar' : 'low'}
delta_time_shape = {'home'   : ['mean', 'low', 'low','mean'],
                    'school' : ['low',	'mean',	'high',	'low'],
                    'restaurant' : ['extralow', 'mean', 'extrahigh', 'mean'],
                    'bar' : ['high', 'high', 'high', 'mean']}
NN = 10
NT = 4
NS = 5
C = 200
random.seed(0)
dfDelta = pd.DataFrame(columns=['j', 't', 's', 'delta'])
dfDemand = pd.DataFrame(columns=['j', 'd'])
for j in range(1, NN +1):
    customer_type = gdf.loc[j]['type']
    dtype = demand_type[customer_type]
    d = C*(random_bounds[dtype][0] + random.random()*(random_bounds[dtype][1] - random_bounds[dtype][0] ))*.7
    d = round(d)
    dfDemand.loc[len(dfDemand)] = [j, d]
    for t in range(1, NT+1):
        delta_type = delta_time_shape[customer_type][t-1]
        for s in range(1, NS + 1):
            delta = C*(random_bounds[delta_type][0] + random.random()*(random_bounds[delta_type][1] - random_bounds[delta_type][0] ))
            delta = round(delta)
            dfDelta.loc[len(dfDelta)] = [j, t, s, delta]
dfDemand.to_excel('real_instance_demand07.xlsx', index=None)
dfDelta.to_excel('real_instance_delta07.xlsx', index=None)



In [8]:
dfDelta = pd.read_excel('real_instance_delta07.xlsx')
dfDelta

,j,t,s,delta
0,1,1,1,35
1,1,1,2,28
2,1,1,3,25
3,1,1,4,30
4,1,1,5,28
...,...,...,...,...
195,10,4,1,51
196,10,4,2,55
197,10,4,3,58
198,10,4,4,50


In [9]:
dfmean = dfDelta.groupby(['j', 't']).agg({'delta' : 'mean'})
dfmean = dfmean.reset_index()
dfmean.insert(2, 's', [1 for i in range(len(dfmean))])
dfmean.to_excel("real_instance_delta_mean07.xlsx", index=None)
dfmean

,j,t,s,delta
0,1,1,1,29.2
1,1,2,1,52.4
2,1,3,1,69.6
3,1,4,1,35.6
4,2,1,1,50.4
5,2,2,1,35.2
6,2,3,1,29.2
7,2,4,1,49.4
8,3,1,1,48.8
9,3,2,1,30.2
